In [1]:
import openai
import time
from openai import OpenAI
from io import BytesIO
from PIL import Image

## Get the API key from .env file

In [2]:
client = OpenAI() # The default API will call os.environ["OPENAI_API_KEY"]

# This if we want to use the .env file's API key
#import dotenv
#client = OpenAI(api_key=dotenv.get_key(".env", 'OPENAI_API_KEY')) 

## Basic prompting
Available models:
1. gpt-3.5-turbo (maximum limit 4097 tokens)
2. dall-e-3
3. whisper-1

In [3]:
# JSON format

response = client.chat.completions.create(
    model = "gpt-3.5-turbo-1106",
    response_format={ "type": "json_object" },
    seed = None, # Optional, allows more deterministic response
    temperature = 1, # Default 1, between 0-2, higher temperatures = more random
    messages=[
    {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
    {"role": "user", "content": "Explain to me how openai migrate works"}
  ]
)

print(response.usage)
print(response.choices[0].message.content)

CompletionUsage(completion_tokens=174, prompt_tokens=30, total_tokens=204)

  {
    "OpenAI Migrate": {
      "Description": "OpenAI Migrate is a tool designed to help users transition from older versions of OpenAI's API to newer versions. It provides guidance and automated suggestions to update code and configurations for compatibility with the latest OpenAI API.",
      "Features": [
        "Automated code and configuration updates",
        "Compatibility checks for older code",
        "Guidance on API changes and best practices"
      ],
      "Usage": {
        "1": "Users can provide their existing code or configuration files to Migrate for analysis",
        "2": "Migrate will then identify areas that need updates and provide suggestions for making the necessary changes",
        "3": "Users can review the suggestions and apply them to their code before transitioning to the latest OpenAI API"
      }
    }
  }


### Multi-response streaming

In [48]:
response = client.chat.completions.create(
    model = "gpt-3.5-turbo-1106",
    stream = True, # event stream like ChatGPT
    seed = None, # Optional, allows more deterministic response
    temperature = 1, # Default 1, between 0-2, higher temperatures = more random
    messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Count from 1 up to 10, answer line by line"}
  ]
)

start_time = time.time()
# create variables to collect the stream of chunks
collected_chunks = []
collected_messages = []
# iterate through the stream of events
for chunk in response:
    chunk_time = time.time() - start_time  # calculate the time delay of the chunk
    collected_chunks.append(chunk)  # save the event response
    chunk_message = chunk.choices[0].delta.content
    if chunk_message:
      collected_messages.append(chunk_message)  # save the message
      print(f"{chunk_time:.2f}s : {chunk_message}")  # print the delay and text

0.00s : 1
0.00s : 

0.00s : 2
0.01s : 

0.01s : 3
0.01s : 

0.01s : 4
0.01s : 

0.01s : 5
0.02s : 

0.02s : 6
0.02s : 

0.02s : 7
0.02s : 

0.02s : 8
0.03s : 

0.03s : 9
0.03s : 

0.03s : 10
Full response received 0.03 seconds after request
Full conversation received: 1
2
3
4
5
6
7
8
9
10


In [50]:
# print the time delay and text received
print(f"Full response received {chunk_time:.2f} seconds after request")
full_reply_content = ''.join(collected_messages)
print(f"Full conversation received: \n{full_reply_content}")

Full response received 0.03 seconds after request
Full conversation received: 
1
2
3
4
5
6
7
8
9
10


## Image

Use LLM to refine basic prompt

In [51]:
response = client.chat.completions.create(
    model = "gpt-3.5-turbo-1106",
    seed = None, # Optional, allows more deterministic response
    temperature = 1, # Default 1, between 0-2, higher temperatures = more random
    messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": '''Refine this prompt to be used in dall-e-3 image generation, add as many descriptive terms as possible: 
     Frontal view of a short-legged and slim maltipoo, 
    it has short curly ivory hair at its head, 
    frizzy hair on its body. It is in living room of a Singapore apartment on a sunny day, 
    the apartment has white parquet flooring, smiling at the sight of its owner'''}
  ]
)

print(response.usage)
print(response.choices[0].message.content)

CompletionUsage(completion_tokens=97, prompt_tokens=107, total_tokens=204)
Generate an image of a charming Maltipoo with short legs and a slender build. The Maltipoo has a frontal view, with short, curly ivory hair on its head and frizzy hair on its body. It is depicted in the living room of a stylish Singapore apartment, illuminated by abundant sunlight. The apartment boasts white parquet flooring, creating a bright and welcoming atmosphere. The adorable Maltipoo is captured smiling at its owner, exuding joy and affection.


In [15]:
maltipoo_prompt = '''Frontal view of a short-legged and slim maltipoo, \
it has short curly ivory hair at its head, \
frizzy hair on its body. It is in living room of a Singapore apartment on a sunny day, \
the apartment has white parquet flooring, smiling at the sight of its owner'''

human_prompt = '''Full body studio shoot of a young king in armour'''

art_prompt = '''Abstract, surreal landscape, musical notes, sun rays'''

instructional_prompt = '''
Generate images, but in a very specific way. Use the exact prompt that is provided without any modifications or rewrites.\n\
Here is what I mean.\n\
This is an example of an exact prompt using comma list: \n\
"A T-Rex with sunglasses on a bicycle, Art Nouveau, High Fashion, 4k, hyper detailed, wide aspect ratio"\n\
Normally you'll rewrite it in something like this and generate the image based on the second prompt: \n\
"A wide 4k hyper-detailed Art Nouveau style illustration showcasing a T-Rex wearing sunglasses, riding a bicycle, all imbued with a touch of high fashion."\n\
I don't want the rewrite. I need an image generated by the exact prompt. I know you can do it.\n\
The first two prompts were just explanation examples.\n\
This is the prompt to work with: \n'''

exact_prompt = ''

final_prompt = instructional_prompt + human_prompt
try:
  response = client.images.generate(
    model="dall-e-3",
    prompt=final_prompt,
    size="1024x1024",
    quality="hd", #standard or hd
    style = 'vivid', #vivid or natural
    n=1
  )

  print(f"Revised prompt: {response.data[0].revised_prompt}")
  image_url = response.data[0].url
  print(image_url)

except openai.OpenAIError as e:
  print(e.http_status)
  print(e.error)

AttributeError: 'BadRequestError' object has no attribute 'http_status'

## Masking
Provide:
1. image (original image)
2. mask (transparent areas as where editing to be done)

In [ ]:
response = client.images.edit(
  model="dall-e-2",
  image=open("ignore/sunlit_lounge.png", "rb"),
  mask=open("ignore/mask.png", "rb"),
  prompt="A sunlit indoor lounge area with a pool containing a flamingo",
  n=1,
  size="1024x1024"
)
image_url = response.data[0].url

## Variations

In [16]:
# Read the image file from disk and resize it
filepath = 'ignore/images/futuristiclink.png'
image = Image.open(filepath)
width, height = 256, 256
image = image.resize((width, height))

# Convert the image to a BytesIO object
byte_stream = BytesIO()
image.save(byte_stream, format='PNG')
byte_array = byte_stream.getvalue()

response = client.images.create_variation(
  image=byte_array,
  n=1,
  size="1024x1024"
)

for i in response.data:
    print(i.url)